# Seasonal-Trend decomposition using LOESS for multiple seasonal components (MSTL)

This note book illustrates the use of `MSTL` [1] to decompose a time series into a: trend component, multiple season components, and a residual component. MSTL uses STL to iteratively extract seasonal components from a time series. The key inputs into `MSTL` are:

* `periods` - The period of each seasonal component. 
* `windows` - The lengths of each seasonal smoother with resect to each period. Must be odd. If `None` a set of default values are chosen from the original paper [1].
* `lmbda` - The lambda parameter for a Box-Cox transformation prior to decomposition. If `None` then no transformation is done. If `"auto"` then an appropriate value for lambda is automatically selected from the data.
* `iterate` - Number of iterations to use to refine the seasonal component.

[1] K. Bandura, R.J. Hyndman, and C. Bergmeir (2021)
    MSTL: A Seasonal-Trend Decomposition Algorithm for Time Series with Multiple
    Seasonal Patterns. arXiv preprint arXiv:2107.13462.
    
Note there are some key differences in this implementation to [1]. Missing data must be handled outside of the `MSTL` class. The algorithm proposed in the paper handles a case when there is no seasonality. This implementation assumes that there is at least one seasonal component.

First we import the required packages, prepare the graphics environment, and prepare the data. 

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

from statsmodels.tsa.seasonal import MSTL, STL

register_matplotlib_converters()
sns.set_style("darkgrid")

In [ ]:
plt.rc("figure", figsize=(16, 12))
plt.rc("font", size=13)

# Create a toy data set with multiple seasonalities

We create a time series with hourly frequency that has a daily and weekly seasonality which follow a sine wave.

In [ ]:
t = np.arange(1, 1000)
daily_seasonality = 5 * np.sin(2 * np.pi * t / 24)
weekly_seasonality = 10 * np.sin(2 * np.pi * t / (24 * 7))
trend = 0.0001 * t**2
y = trend + daily_seasonality + weekly_seasonality + np.random.randn(len(t))
ts = pd.date_range(start="2020-01-01", freq="H", periods=len(t))
df = pd.DataFrame(data=y, index=ts, columns=["y"])

In [ ]:
df.head()

Let's plot the time series

In [ ]:
df["y"].plot(figsize=[10, 5])

# Apply decomposition

In [ ]:
mstl = MSTL(df["y"], periods=[24, 24 * 7])
res = mstl.fit()

If the input is a pandas dataframe then the output for the seasonal component is a dataframe. The period for each component is reflect in the column names.

In [ ]:
res.seasonal.head()

In [ ]:
ax = res.plot()

We see that the hourly and weekly seasonal components have been extracted.

Any of the STL parameters other than `period` and `seasonal` (as they are set by `periods` and `windows` in `MSTL`) can also be set by passing arg:value pairs as a dictionary to `stl_kwargs`.

Here we show that we can still set the trend smoother of STL via `trend`, although it will result in a worse fit. We will also explicitly set the `windows`, `seasonal_deg`, and `iterate` parameter.

In [ ]:
mstl = MSTL(
    df,
    periods=[24, 24 * 7],  # The periods and windows must be the same length and will correspond to one another
    windows=[101, 101],  # Setting this large along with `seasonal_deg=0` will force the seasonality to be periodic
    iterate=3,
    stl_kwargs={
                'trend':1001, # Setting this large will force the trend to be smoother
                'seasonal_deg':0 # Means the seasonal smoother is fit with a moving average
               }
)
res = mstl.fit()
ax = res.plot()